In [1]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import ast
import numpy as np

In [2]:
wiki_movies = pd.read_csv("wikipedia_movie_franchises.csv")
imdb_movies = pd.read_csv("imdb_movies_db.csv")

In [3]:
wiki_movies = wiki_movies.drop("Unnamed: 0", axis = 1)
wiki_movies

,franchise_id,franchise_name,movie_name,release_year
0,f0,The Aldrich Family,What a Life,1949.0
1,f0,The Aldrich Family,Life with Henry,1952.0
2,f0,The Aldrich Family,Henry Aldrich for President,1952.0
3,f0,The Aldrich Family,"Henry Aldrich, Editor",1952.0
4,f0,The Aldrich Family,Henry and Dizzy,1952.0
...,...,...,...,...
8225,f1342,Wong Fei Hung (Chinese films),Huang Fei-hong xi lie zhi yi dai shi,1992.0
8226,f1342,Wong Fei Hung (Chinese films),Huang Fei Hong zhi nan er dang bao guo,1993.0
8227,f1342,Wong Fei Hung (Chinese films),Huang Fei Hong dui Huang Fei Hong,1993.0
8228,f1342,Wong Fei Hung (Chinese films),Huang Fei-Hong zhi gui jiao qi,1993.0


In [4]:
imdb_movies = imdb_movies.drop("Unnamed: 0", axis = 1)
imdb_movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance,5.3,205.0,"['Miss Jerry', 'Fräulein Jerry']"
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,100,"Documentary,News,Sport",5.3,469.0,"['The Corbett-Fitzsimmons Fight', 'Бой Корбетт..."
2,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,4.1,15.0,['Bohemios']
3,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography",6.0,823.0,"['The Story of the Kelly Gang', 'Kelly bandájá..."
4,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama,4.4,20.0,"[""L'enfant prodigue"", 'The Prodigal Son']"
...,...,...,...,...,...,...,...,...,...,...,...,...
290234,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,\N,84,Thriller,5.8,1414.0,"['Il talento del calabrone', 'The talent of th..."
290235,tt9916362,movie,Coven,Akelarre,0,2020,\N,92,"Drama,History",6.4,5293.0,"['Boszorkánygyülekezet', 'Coven', ""Les sorcièr..."
290236,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019,\N,\N,"Adventure,History,War",3.8,14.0,"['The Secret of China', 'Hong xing zhao yao Zh..."
290237,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,\N,123,Drama,8.6,7.0,['Kuambil Lagi Hatiku']


In [5]:
wiki_movies = wiki_movies.dropna(axis=0, how='any')
wiki_movies.reset_index(inplace = True)
wiki_movies["release_year"] = wiki_movies["release_year"].apply(lambda x : str(int(x)))

C:\Users\35387\AppData\Local\Temp\ipykernel_13976\4040367188.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wiki_movies["release_year"] = wiki_movies["release_year"].apply(lambda x : str(int(x)))


In [6]:
wiki_movies["movie_name"] = wiki_movies["movie_name"].apply(lambda x : x.strip())
wiki_movies["franchise_name"] = wiki_movies["franchise_name"].apply(lambda x : x.strip())

C:\Users\35387\AppData\Local\Temp\ipykernel_13976\799754025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wiki_movies["movie_name"] = wiki_movies["movie_name"].apply(lambda x : x.strip())
C:\Users\35387\AppData\Local\Temp\ipykernel_13976\799754025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wiki_movies["franchise_name"] = wiki_movies["franchise_name"].apply(lambda x : x.strip())


In [7]:
wiki_movies["movie_name"] = wiki_movies["movie_name"].apply(lambda x : x.replace("–", "-"))
wiki_movies["movie_name"] = wiki_movies["movie_name"].apply(lambda x : x.lower())

C:\Users\35387\AppData\Local\Temp\ipykernel_13976\2671306462.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wiki_movies["movie_name"] = wiki_movies["movie_name"].apply(lambda x : x.replace("–", "-"))
C:\Users\35387\AppData\Local\Temp\ipykernel_13976\2671306462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wiki_movies["movie_name"] = wiki_movies["movie_name"].apply(lambda x : x.lower())


In [8]:
def lower_akas(row):
    if type(row) == str:
        row = row.lower()
        return row

In [9]:
imdb_movies["primaryTitle"] = imdb_movies["primaryTitle"].apply(lambda x : x.lower())
imdb_movies["originalTitle"] = imdb_movies["originalTitle"].apply(lambda x : x.lower())
imdb_movies["akas"] = imdb_movies["akas"].apply(lambda x : lower_akas(x))

In [49]:
wiki_filter = ["tconst", "titleType", "primaryTitle","originalTitle", "isAdult","startYear","endYear","runtimeMinutes","genres","averageRating","numVotes","akas","_merge", "all_names"]
imdb_filter = ["index", "franchise_name", "franchise_id", "movie_name", "release_year","_merge"]

In [77]:
imdb_copy = imdb_movies.copy()

In [78]:
def filter_df(df, merge, cols):
    remainder = df[df["_merge"]== merge]
    remainder = remainder.drop(cols, axis = 1)
    return remainder

In [79]:
def extract_akas(row):
    # convert string in list format to list
    if type(row) == str:
        akas = ast.literal_eval(row)
        return akas

In [80]:
imdb_copy["akas"] = imdb_copy["akas"].apply(lambda x : extract_akas(x))

In [81]:
imdb_copy['all_names'] = np.empty((len(imdb_copy), 0)).tolist()

In [82]:
for i in range(0, len(list(imdb_copy["primaryTitle"]))):
    imdb_copy['all_names'][i].append(imdb_copy['primaryTitle'][i])
    imdb_copy['all_names'][i].append(imdb_copy['originalTitle'][i])
    
    if type(imdb_copy['akas'][i]) == list:
        for x in imdb_copy['akas'][i]:
            imdb_copy['all_names'][i].append(x)
        
        
imdb_copy

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names
0,tt0000009,movie,miss jerry,miss jerry,0,1894,\N,45,Romance,5.3,205.0,"[miss jerry, fräulein jerry]","[miss jerry, miss jerry, miss jerry, fräulein ..."
1,tt0000147,movie,the corbett-fitzsimmons fight,the corbett-fitzsimmons fight,0,1897,\N,100,"Documentary,News,Sport",5.3,469.0,"[the corbett-fitzsimmons fight, бой корбетта и...","[the corbett-fitzsimmons fight, the corbett-fi..."
2,tt0000502,movie,bohemios,bohemios,0,1905,\N,100,\N,4.1,15.0,[bohemios],"[bohemios, bohemios, bohemios]"
3,tt0000574,movie,the story of the kelly gang,the story of the kelly gang,0,1906,\N,70,"Action,Adventure,Biography",6.0,823.0,"[the story of the kelly gang, kelly bandájának...","[the story of the kelly gang, the story of the..."
4,tt0000591,movie,the prodigal son,l'enfant prodigue,0,1907,\N,90,Drama,4.4,20.0,"[l'enfant prodigue, the prodigal son]","[the prodigal son, l'enfant prodigue, l'enfant..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290234,tt9916270,movie,il talento del calabrone,il talento del calabrone,0,2020,\N,84,Thriller,5.8,1414.0,"[il talento del calabrone, the talent of the h...","[il talento del calabrone, il talento del cala..."
290235,tt9916362,movie,coven,akelarre,0,2020,\N,92,"Drama,History",6.4,5293.0,"[boszorkánygyülekezet, coven, les sorcières d'...","[coven, akelarre, boszorkánygyülekezet, coven,..."
290236,tt9916428,movie,the secret of china,hong xing zhao yao zhong guo,0,2019,\N,\N,"Adventure,History,War",3.8,14.0,"[the secret of china, hong xing zhao yao zhong...","[the secret of china, hong xing zhao yao zhong..."
290237,tt9916538,movie,kuambil lagi hatiku,kuambil lagi hatiku,0,2019,\N,123,Drama,8.6,7.0,[kuambil lagi hatiku],"[kuambil lagi hatiku, kuambil lagi hatiku, kua..."


In [83]:
imdb_copy = imdb_copy.explode("all_names")

In [84]:
all_merged = pd.merge(wiki_movies, imdb_copy, left_on=["movie_name", "release_year"], right_on=["all_names", "startYear"], how="outer", indicator = True)
wiki_remainder = filter_df(all_merged, "left_only", wiki_filter)
imdb_remainder = filter_df(all_merged, "right_only", imdb_filter)
all_merged[all_merged["_merge"]=="both"]

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
12,12.0,f1,Coffin Joe,this night i'll possess your corpse,1967,tt0060380,movie,esta noite encarnarei no teu cadáver,esta noite encarnarei no teu cadáver,0.0,1967,\N,108,Horror,7.0,1952.0,"[esta noche poseeré tu cadáver, esta noite enc...",this night i'll possess your corpse,both
13,13.0,f1,Coffin Joe,the strange world of coffin joe,1968,tt0062943,movie,the strange world of coffin joe,o estranho mundo de zé do caixão,0.0,1968,\N,80,Horror,6.3,653.0,"[странный мир зе-из-гроба, the strange world o...",the strange world of coffin joe,both
14,13.0,f1,Coffin Joe,the strange world of coffin joe,1968,tt0062943,movie,the strange world of coffin joe,o estranho mundo de zé do caixão,0.0,1968,\N,80,Horror,6.3,653.0,"[странный мир зе-из-гроба, the strange world o...",the strange world of coffin joe,both
15,14.0,f1,Coffin Joe,awakening of the beast,1970,tt0062690,movie,awakening of the beast,o ritual dos sádicos,0.0,1970,\N,93,"Drama,Horror",5.8,1208.0,"[пробуждение зверя, ritual of the maniacs, o d...",awakening of the beast,both
16,14.0,f1,Coffin Joe,awakening of the beast,1970,tt0062690,movie,awakening of the beast,o ritual dos sádicos,0.0,1970,\N,93,"Drama,Horror",5.8,1208.0,"[пробуждение зверя, ritual of the maniacs, o d...",awakening of the beast,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16296,8226.0,f1342,Wong Fei Hung (Chinese films),huang fei hong zhi nan er dang bao guo,1993,tt0107161,movie,fist from shaolin,huang fei hong zhi nan er dang bao guo,0.0,1993,\N,94,"Action,Adventure,Comedy",6.5,95.0,"[wong fei hung ji naam yi dong bo gwok, huang ...",huang fei hong zhi nan er dang bao guo,both
16297,8227.0,f1342,Wong Fei Hung (Chinese films),huang fei hong dui huang fei hong,1993,tt0107160,movie,huang fei hong dui huang fei hong,huang fei hong dui huang fei hong,0.0,1993,\N,88,"Action,Comedy,Crime",6.0,144.0,"[wong fei hung dui wong fei hung, master wong ...",huang fei hong dui huang fei hong,both
16298,8227.0,f1342,Wong Fei Hung (Chinese films),huang fei hong dui huang fei hong,1993,tt0107160,movie,huang fei hong dui huang fei hong,huang fei hong dui huang fei hong,0.0,1993,\N,88,"Action,Comedy,Crime",6.0,144.0,"[wong fei hung dui wong fei hung, master wong ...",huang fei hong dui huang fei hong,both
16299,8227.0,f1342,Wong Fei Hung (Chinese films),huang fei hong dui huang fei hong,1993,tt0107160,movie,huang fei hong dui huang fei hong,huang fei hong dui huang fei hong,0.0,1993,\N,88,"Action,Comedy,Crime",6.0,144.0,"[wong fei hung dui wong fei hung, master wong ...",huang fei hong dui huang fei hong,both


In [85]:
all_merged_both = all_merged[all_merged["_merge"]=="both"]
all_merged_both = all_merged_both.drop_duplicates(subset = ["index", "tconst"],keep = "first")
all_merged_both

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
12,12.0,f1,Coffin Joe,this night i'll possess your corpse,1967,tt0060380,movie,esta noite encarnarei no teu cadáver,esta noite encarnarei no teu cadáver,0.0,1967,\N,108,Horror,7.0,1952.0,"[esta noche poseeré tu cadáver, esta noite enc...",this night i'll possess your corpse,both
13,13.0,f1,Coffin Joe,the strange world of coffin joe,1968,tt0062943,movie,the strange world of coffin joe,o estranho mundo de zé do caixão,0.0,1968,\N,80,Horror,6.3,653.0,"[странный мир зе-из-гроба, the strange world o...",the strange world of coffin joe,both
15,14.0,f1,Coffin Joe,awakening of the beast,1970,tt0062690,movie,awakening of the beast,o ritual dos sádicos,0.0,1970,\N,93,"Drama,Horror",5.8,1208.0,"[пробуждение зверя, ritual of the maniacs, o d...",awakening of the beast,both
18,16.0,f1,Coffin Joe,the bloody exorcism of coffin joe,1974,tt0071479,movie,exorcismo negro,exorcismo negro,0.0,1974,\N,100,Horror,6.0,274.0,"[exorcisme noir, black exorcism of coffin joe,...",the bloody exorcism of coffin joe,both
19,17.0,f1,Coffin Joe,the strange hostel of naked pleasures,1976,tt0074499,movie,the strange hostel of naked pleasures,a estranha hospedaria dos prazeres,0.0,1976,\N,81,Horror,4.7,289.0,"[a estranha hospedaria dos prazeres, the stran...",the strange hostel of naked pleasures,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16287,8221.0,f1342,Wong Fei Hung (Chinese films),martial club,1981,tt0083343,movie,martial club,wu guan,0.0,1981,\N,110,"Action,Drama",7.0,580.0,"[o clube das artes marciais, el club marcial d...",martial club,both
16291,8224.0,f1342,Wong Fei Hung (Chinese films),huang fei hong xiao zhuan,1992,tt0104459,movie,huang fei hong xiao zhuan,huang fei hong xiao zhuan,0.0,1992,\N,93,"Action,Comedy,Crime",5.9,184.0,"[wong fei hung siu chuen, huang fei hong xiao ...",huang fei hong xiao zhuan,both
16295,8226.0,f1342,Wong Fei Hung (Chinese films),huang fei hong zhi nan er dang bao guo,1993,tt0107161,movie,fist from shaolin,huang fei hong zhi nan er dang bao guo,0.0,1993,\N,94,"Action,Adventure,Comedy",6.5,95.0,"[wong fei hung ji naam yi dong bo gwok, huang ...",huang fei hong zhi nan er dang bao guo,both
16297,8227.0,f1342,Wong Fei Hung (Chinese films),huang fei hong dui huang fei hong,1993,tt0107160,movie,huang fei hong dui huang fei hong,huang fei hong dui huang fei hong,0.0,1993,\N,88,"Action,Comedy,Crime",6.0,144.0,"[wong fei hung dui wong fei hung, master wong ...",huang fei hong dui huang fei hong,both


In [86]:
no_year = pd.merge(wiki_movies, imdb_remainder, left_on=["movie_name"], right_on=["all_names"], how="outer", indicator = True)
wiki_remainder = filter_df(all_merged, "left_only", wiki_filter)
imdb_remainder = filter_df(all_merged, "right_only", imdb_filter)
no_year

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
0,0.0,f0,The Aldrich Family,what a life,1949,tt0032123,movie,what a life,what a life,0.0,1939,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both
1,0.0,f0,The Aldrich Family,what a life,1949,tt0032123,movie,what a life,what a life,0.0,1939,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both
2,0.0,f0,The Aldrich Family,what a life,1949,tt0032123,movie,what a life,what a life,0.0,1939,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both
3,1.0,f0,The Aldrich Family,life with henry,1952,tt0033834,movie,life with henry,life with henry,0.0,1940,\N,80,"Comedy,Family,Music",6.0,49.0,"[life with henry, henry quería ir a alaska, he...",life with henry,both
4,1.0,f0,The Aldrich Family,life with henry,1952,tt0033834,movie,life with henry,life with henry,0.0,1940,\N,80,"Comedy,Family,Music",6.0,49.0,"[life with henry, henry quería ir a alaska, he...",life with henry,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1911601,NaN,NaN,NaN,NaN,NaN,tt9916538,movie,kuambil lagi hatiku,kuambil lagi hatiku,0.0,2019,\N,123,Drama,8.6,7.0,[kuambil lagi hatiku],kuambil lagi hatiku,right_only
1911602,NaN,NaN,NaN,NaN,NaN,tt9916730,movie,6 gunn,6 gunn,0.0,2017,\N,116,\N,8.3,10.0,"[६ गुण, 6 gunn]",6 gunn,right_only
1911603,NaN,NaN,NaN,NaN,NaN,tt9916730,movie,6 gunn,6 gunn,0.0,2017,\N,116,\N,8.3,10.0,"[६ गुण, 6 gunn]",6 gunn,right_only
1911604,NaN,NaN,NaN,NaN,NaN,tt9916730,movie,6 gunn,6 gunn,0.0,2017,\N,116,\N,8.3,10.0,"[६ गुण, 6 gunn]",6 gunn,right_only


In [88]:
no_year_both = no_year[no_year["_merge"]=="both"]
no_year_both = no_year_both.drop_duplicates(subset = ["index", "tconst"],keep = "first")
no_year_both

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
0,0.0,f0,The Aldrich Family,what a life,1949,tt0032123,movie,what a life,what a life,0.0,1939,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both
3,1.0,f0,The Aldrich Family,life with henry,1952,tt0033834,movie,life with henry,life with henry,0.0,1940,\N,80,"Comedy,Family,Music",6.0,49.0,"[life with henry, henry quería ir a alaska, he...",life with henry,both
6,2.0,f0,The Aldrich Family,henry aldrich for president,1952,tt0033708,movie,henry aldrich for president,henry aldrich for president,0.0,1941,\N,75,"Comedy,Family",6.6,146.0,"[henry aldrich para presidente, henry aldrich ...",henry aldrich for president,both
9,3.0,f0,The Aldrich Family,"henry aldrich, editor",1952,tt0034842,movie,"henry aldrich, editor","henry aldrich, editor",0.0,1942,\N,72,"Comedy,Drama,Family",6.4,150.0,"[henry periodista, henry aldrich, editor]","henry aldrich, editor",both
12,4.0,f0,The Aldrich Family,henry and dizzy,1952,tt0034844,movie,henry and dizzy,henry and dizzy,0.0,1942,\N,71,"Comedy,Family",7.2,58.0,[henry and dizzy],henry and dizzy,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13208,8068.0,f1340,Hopalong Cassidy (American-Western),fool's gold,1947,tt0770752,movie,fool's gold,fool's gold,0.0,2008,\N,112,"Action,Adventure,Comedy",5.7,81773.0,"[fool's gold, chasse au trésor, amor y tesoro,...",fool's gold,both
13211,8068.0,f1340,Hopalong Cassidy (American-Western),fool's gold,1947,tt0891302,movie,fool's gold,fool's gold,0.0,2003,\N,55,Documentary,5.8,24.0,[fool's gold],fool's gold,both
13217,8072.0,f1340,Hopalong Cassidy (American-Western),the marauders,1947,tt0048351,movie,the marauders,the marauders,0.0,1955,\N,81,Western,5.8,219.0,"[tuhopolttajat, los merodeadores, the marauder...",the marauders,both
13279,8132.0,f1341,Super Sentai,doubutsu sentai zyuohger vs. ninninger the mov...,2016,tt6342418,movie,doubutsu sentai zyuohger vs. ninninger the mov...,gekijôban dôbutsu sentai jûôjâ tai ninninjâ mi...,0.0,2017,\N,64,Action,6.9,35.0,[gekijôban dôbutsu sentai jûôjâ tai ninninjâ m...,doubutsu sentai zyuohger vs. ninninger the mov...,both


In [89]:
matched = pd.concat([all_merged_both, no_year_both])
matched = matched[matched["_merge"]=="both"]
matched.sort_values("numVotes", ascending = False).drop_duplicates(subset=["index", "franchise_id"], keep="first").sort_values(["index"])
matched

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
12,12.0,f1,Coffin Joe,this night i'll possess your corpse,1967,tt0060380,movie,esta noite encarnarei no teu cadáver,esta noite encarnarei no teu cadáver,0.0,1967,\N,108,Horror,7.0,1952.0,"[esta noche poseeré tu cadáver, esta noite enc...",this night i'll possess your corpse,both
13,13.0,f1,Coffin Joe,the strange world of coffin joe,1968,tt0062943,movie,the strange world of coffin joe,o estranho mundo de zé do caixão,0.0,1968,\N,80,Horror,6.3,653.0,"[странный мир зе-из-гроба, the strange world o...",the strange world of coffin joe,both
15,14.0,f1,Coffin Joe,awakening of the beast,1970,tt0062690,movie,awakening of the beast,o ritual dos sádicos,0.0,1970,\N,93,"Drama,Horror",5.8,1208.0,"[пробуждение зверя, ritual of the maniacs, o d...",awakening of the beast,both
18,16.0,f1,Coffin Joe,the bloody exorcism of coffin joe,1974,tt0071479,movie,exorcismo negro,exorcismo negro,0.0,1974,\N,100,Horror,6.0,274.0,"[exorcisme noir, black exorcism of coffin joe,...",the bloody exorcism of coffin joe,both
19,17.0,f1,Coffin Joe,the strange hostel of naked pleasures,1976,tt0074499,movie,the strange hostel of naked pleasures,a estranha hospedaria dos prazeres,0.0,1976,\N,81,Horror,4.7,289.0,"[a estranha hospedaria dos prazeres, the stran...",the strange hostel of naked pleasures,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13208,8068.0,f1340,Hopalong Cassidy (American-Western),fool's gold,1947,tt0770752,movie,fool's gold,fool's gold,0.0,2008,\N,112,"Action,Adventure,Comedy",5.7,81773.0,"[fool's gold, chasse au trésor, amor y tesoro,...",fool's gold,both
13211,8068.0,f1340,Hopalong Cassidy (American-Western),fool's gold,1947,tt0891302,movie,fool's gold,fool's gold,0.0,2003,\N,55,Documentary,5.8,24.0,[fool's gold],fool's gold,both
13217,8072.0,f1340,Hopalong Cassidy (American-Western),the marauders,1947,tt0048351,movie,the marauders,the marauders,0.0,1955,\N,81,Western,5.8,219.0,"[tuhopolttajat, los merodeadores, the marauder...",the marauders,both
13279,8132.0,f1341,Super Sentai,doubutsu sentai zyuohger vs. ninninger the mov...,2016,tt6342418,movie,doubutsu sentai zyuohger vs. ninninger the mov...,gekijôban dôbutsu sentai jûôjâ tai ninninjâ mi...,0.0,2017,\N,64,Action,6.9,35.0,[gekijôban dôbutsu sentai jûôjâ tai ninninjâ m...,doubutsu sentai zyuohger vs. ninninger the mov...,both


In [90]:
len(matched["franchise_id"].unique())

1169

In [91]:
matched.to_csv("movies_with_franchises.csv")

# Unused code

In [46]:
list1 = list(matched["index"])
found_mask = []
for i in range(0, len(list(wiki_movies["index"]))):
    if i in list1:
        found_mask.append(True)
    else:
        found_mask.append(False)
    
#print(list1)
print(wiki_movies[found_mask])

      index franchise_id                 franchise_name  \
12       12           f1                     Coffin Joe   
13       13           f1                     Coffin Joe   
14       14           f1                     Coffin Joe   
16       16           f1                     Coffin Joe   
17       17           f1                     Coffin Joe   
...     ...          ...                            ...   
8122   8153        f1342  Wong Fei Hung (Chinese films)   
8127   8158        f1342  Wong Fei Hung (Chinese films)   
8128   8159        f1342  Wong Fei Hung (Chinese films)   
8141   8172        f1342  Wong Fei Hung (Chinese films)   
8144   8175        f1342  Wong Fei Hung (Chinese films)   

                                 movie_name release_year  
12      this night i'll possess your corpse         1967  
13          the strange world of coffin joe         1968  
14                   awakening of the beast         1970  
16        the bloody exorcism of coffin joe         197

In [ ]:
# apply this mask function at each step of the matching process and see how many found rows there are

In [47]:
3450 + 4749

8199

In [48]:
matched["index"].value_counts()

12.0      1
5385.0    1
5382.0    1
5381.0    1
5380.0    1
         ..
7352.0    1
2650.0    1
7351.0    1
2649.0    1
8228.0    1
Name: index, Length: 4760, dtype: int64

In [ ]:
both_test = matched[matched["_merge"]=="both"].sort_values(by = "index")

In [ ]:
dict(both_test["franchise_name"].value_counts())

In [ ]:
# look into moving all names into one list, then searching with date
# With remainder, search without date and see what happens

In [ ]:
# losing some values in both data frames for unknown reason when using merge.
# starts with 6405 rows in wiki and 290239 rows in imdb
# 3307 
#              wiki  |  imdb 
# start     |  6405  |  290239
# found     |  3088  |    3088
# remaining |  3307  |  287199
# total     |  6395  |  290287
# variance  |   -10  |     +48

### Checks for movies in dataframes

In [ ]:
test = both_test[both_test["franchise_name"].str.contains("Edgar Wallace Mysteries", na=False, flags=re.IGNORECASE, regex=True)]
test.head(50)

In [ ]:
test = wiki_movies[wiki_movies["franchise_name"].str.contains("Edgar Wallace Mysteries", na=False, flags=re.IGNORECASE, regex=True)]
test

In [ ]:
test = imdb_movies[imdb_movies["primaryTitle"].str.contains("star wars", na=False, flags=re.IGNORECASE, regex=True)]
test

In [ ]:
test = matched[matched["index"] == 5191.0]
test

In [ ]:
test = merged_df[merged_df["index"].str.contains("star wars", na=False, flags=re.IGNORECASE, regex=True)]
test


In [ ]:
imdb_movies['titles'] = imdb_movies.apply(', '.join(imdb_movies.columns["primaryTitle", "originalTitle", "akas"]), axis = 1)
imdb_movies

In [ ]:
def fuzzy_match(wiki, imdb, threshold):
    # iterate through each row in wiki
    # 
    # 
    # 
    # 
    # 
    # 
    # 

In [ ]:
fuzzy_match(wiki_movies, imdb_movies, 95)